Machine learning has been existing for decades. Before the prevalence of doing machine learning with Python, many other languages such as Java, C++ were used to build models. Refactoring legacy models in C++ or Java could be forbiddingly expensive and time consuming. In this blog, we are going demonstrate inferencing C++ models by building a customer container first and then run a ScriptProcessor jobs. 

# C++ model

We use a simple C++ test file for demonstration purpose. This C++ program accepts input data as a series of strings separated by comma. For example, “2,3“ represent a row of input data 2 and 3 in two separate columns. Each row has two column. The C++ code will parse out numeric values and run inference. We use a simple linear regression model y=x1 + x2 in this example. Customer can modify the C++ inference code to inference more realistic and complicated models. The C++ program will print out result (i.e., y in this case) to standard output stream. The complete Python script is shown below:

In [ ]:
!pygmentize test.cpp

We use `g++` to compile the `test.cpp` file into an executable file `a.out`

In [ ]:
!g++ -std=c++11 test.cpp

We run a quick test on `a.out` to make sure it works as expected.

In [ ]:
%%sh
./a.out '9,8'

# SageMaker Processing

Amazon SageMaker Processing is a new capability of Amazon SageMaker (https://aws.amazon.com/sagemaker/) for running processing and model evaluation workloads with a fully managed experience. Amazon SageMaker Processing lets customers run analytics jobs for data engineering and model evaluation on Amazon SageMaker easily and at scale. SageMaker Processing allows customers to enjoy the benefits of a fully managed environment with all the security and compliance guarantees built into Amazon SageMaker. With Amazon SageMaker Processing, customers have the flexibility of using the built-in data processing containers or bringing their own containers and submitting custom jobs to run on managed infrastructure. Once submitted, Amazon SageMaker launches the compute instances, processes and analyzes the input data and releases the resources upon completion. 

The processing container is defined as shown below. We have Anaconda and Pandas installed into the container. `a.out` is the C++ executable that contains the model inference logic. `processing.py` is the Python script we use to call C++ executable and save results. We build the Docker container and push it to Amazon Elastic Container Registry. 

### Build container

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=cpp_processing

#cd container

chmod +x process_script.py
chmod +x a.out

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

### SageMaker Processing script

Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the container that was just built, and `process_script.py` script for calling the C++ model.

The `process_script.py` first finds all data files under `/opt/ml/processing/input/`. These data files are downloaded by SageMaker from S3 to designated local directory in the container. By default, when you use multiple instances, data from S3 are duplicated to each container instance. That means every instance get the full dataset. By setting `s3_data_distribution_type='ShardedByS3Key'`, each instance gets approximately 1/n of the number of total input date files. 

We read each data file into memory and convert it into a long string ready for C++ executable to consume. `subprocess` module from Python allows us to run the C++ executable and connect to output and error pipes. Output is saved as csv file to `/opt/ml/processing/output`. Upon completion, SageMaker Processing will upload files in this directory to S3. The main script looks like below:
```python
def call_one_exe(a):
    p = subprocess.Popen(["./a.out", a],stdout=subprocess.PIPE)
    p_out, err= p.communicate()
    output = p_out.decode("utf-8")
    return output.split(',')

if __name__=='__main__':
    #parse is only needed if we want to pass arg
    parser = argparse.ArgumentParser()
    #parser.add_argument('--num_thread', type=int, default=1000)
    args, _ = parser.parse_known_args()
    
    print('Received arguments {}'.format(args))
    
    files = glob('/opt/ml/processing/input/*')
    
    for i, f in enumerate(files):
        try:
            data = pd.read_csv(f, header=None)
            string = str(list(data.values.flat)).replace(' ','')[1:-1]
            #string looks like 2,3,5,6,7,8. Space is removed. '[' and ']' are also removed.
            predictions = call_one_exe(string)
            
            output_path = os.path.join('/opt/ml/processing/output', str(i)+'_out.csv')
            print('Saving training features to {}'.format(output_path))
            pd.DataFrame({'results':predictions}).to_csv(output_path, header=False, index=False)
        except Exception as e:
            print(str(e))
            
```


### Run a processing job 

The next step would be to configure a processing job using the ScriptProcessor object.

In [ ]:
import boto3, os
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
default_s3_bucket = sagemaker_session.default_bucket()


client = boto3.client('sts')
Account_number = client.get_caller_identity()['Account']


10 sample data files are included in this demo. Each file contains 5000 raws of arbitrarily generated data. We first upload these 200 files to S3.

In [ ]:
input_data = sagemaker_session.upload_data(path='./data_files', 
                                           bucket=default_s3_bucket, 
                                           key_prefix='data_for_inference_with_cpp_model')

Run a processing job using the Docker image and preprocessing script you just created. We pass the Amazon S3 input and output paths as arguments that are required by our preprocessing script to determine input and output location in Amazon S3. Here, we also specify the number of instances and instance type that will be used for the processing job.

In [ ]:
role = get_execution_role()
script_processor = ScriptProcessor(command=['python3'],
                image_uri=Account_number + '.dkr.ecr.us-east-1.amazonaws.com/cpp_processing:latest',
                role=role,
                instance_count=1,
                base_job_name = 'run-exe-processing',
                instance_type='ml.c5.xlarge')

In [ ]:
output_location = os.path.join('s3://',default_s3_bucket, 'processing_output')

script_processor.run(code='process_script.py',
                     inputs=[ProcessingInput(
                        source=input_data,
                        destination='/opt/ml/processing/input')],
                      outputs=[ProcessingOutput(source='/opt/ml/processing/output',
                                               destination=output_bucket)]
                    )

### Inspect the preprocessed dataset
Take a look at a few rows of one dataset to make sure the preprocessing was successful.

In [ ]:
print('Top 5 rows from 1_out.csv')
!aws s3 cp $output_location/0_out.csv - | head -n5